In [54]:
from netCDF4 import Dataset

import numpy as np

import metpy.calc as mpcalc
from metpy.units import units
from metpy.plots import current_weather, sky_cover, StationPlot, simple_layout
from metpy.io import metar
from metpy.plots import wx_code_to_numeric

import pandas as pd

import matplotlib.pyplot as plt

import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [55]:
data = Dataset ("./20221212_0000")

In [56]:
stationName = data.variables["stationName"][:]
latitude = data.variables["latitude"][:]
longitude = data.variables["longitude"][:]
temp = data.variables["temperature"][:]
dew = data.variables["dewpoint"][:]
windSpeed = data.variables["windSpeed"][:]
windDir = data.variables["windDir"][:]
gust = data.variables["windGust"][:]
weather = data.variables["presWeather"][:]
vis = data.variables["visibility"][:]
slPress = data.variables["seaLevelPress"][:]/100
skyCover = data.variables['skyCover'][:]

In [57]:
stnNames = [""]*len(stationName)
#print(stnNames)
for i in range(len (stationName)):
    stn = stationName[i].tobytes().decode().rstrip("\x00")
    stnNames[i] = stn


curWx = [""]*len(weather)
for i in range(len(weather)):
    wx = weather[i].tobytes().decode().rstrip("\x00")
    curWx[i] = wx
wxCodes = wx_code_to_numeric(curWx)

#print(stnNames)
skyCoverStr = ['']*len(skyCover)
skyCvrNum = np.empty(len(skyCover))
for i in range(len(skyCover)):
    thisSkyCvr = skyCover[i].tobytes().decode().rstrip("\x00")
    skyCoverStr[i] = thisSkyCvr

    if thisSkyCvr[-3:] == 'SKC' or thisSkyCvr[-3:] == 'CLR':
        skyCvrNum[i] = 0
    elif thisSkyCvr[-3:] == 'FEW':
        skyCvrNum[i] = 2
    elif thisSkyCvr[-3:] == 'SCT':
        skyCvrNum[i] = 4
    elif thisSkyCvr[-3:] == 'BKN':
        skyCvrNum[i] = 6 
    elif thisSkyCvr[-3:] == 'OVC':
        skyCvrNum[i] = 8
    else:
        skyCvrNum[i] = np.nan

    print(skyCoverStr[i])




CLR
BKN
BKN
SCT     BKN     OVC
VV
SCT     BKN     BKN     SCT
FEW     SCT     BKN     FEW
SCT     SCT     BKN




BKN
SCT


FEW     BKN
SCT     BKN
BKN     FEW
FEW     SCT


FEW
FEW


SCT     SCT     BKN
SCT
SCT     FEW     BKN


SCT
FEW     SCT     BKN     BKN
SCT
SCT
FEW
SCT     SCT     FEW     BKN
FEW

FEW     SCT
SCT     BKN
SCT
BKN
FEW     SCT
FEW


FEW


FEW     SCT
SCT
FEW     SCT
BKN     BKN     BKN
FEW     SCT
BKN     OVC
SCT     BKN     BKN
SCT     BKN     OVC
SCT
FEW     SCT

SCT     SCT
FEW
BKN
SCT     BKN
FEW     FEW     FEW     SCT
FEW
SCT
CLR
FEW     FEW
SCT
SCT     BKN
FEW
FEW
SCT
FEW
CLR
CLR
FEW     BKN     BKN
OVC
FEW     BKN
SCT     OVC
CLR
BKN     OVC

CLR
CLR



CLR
CLR
CLR
CLR
FEW     SCT
FEW     FEW     OVC
SCT     BKN     OVC
FEW     SCT     OVC
FEW     SCT     OVC
FEW     OVC
BKN     OVC
BKN     OVC
FEW     BKN     OVC
CLR
CLR
CLR
FEW     BKN     BKN     BKN
FEW     BKN     OVC
BKN     OVC
CLR
FEW     OVC
FEW
OVC
FEW     OVC
CLR
SCT     BKN     OVC

CLR
FEW   

In [58]:
dataForDf = {'station_id':stnNames, 'latitude':latitude,
'longitude':longitude,'wind_direction' :windDir,'wind_speed': windSpeed,  'wind_gust':gust, 'visibility':vis, 'current_wx1_symbol':wxCodes,
'air_temperature':temp, 'dew_point_temperature':dew,'air_pressure_at_sea_level':slPress }#'eastwardWind':u, 'northwardWind':v


df = pd.DataFrame (dataForDf, index = stnNames)




In [60]:
#first convert the lat/lon to map projection coordinates
locs = proj.transform_points(ccrs.PlateCarree(),df['longitude'].values,df['latitude'].values)

#now thin the data
thinData = df[mpcalc.reduce_point_density(locs, 250000)]
rethinnedData = df[mpcalc.reduce_point_density(locs, 90000)]


#map projection
proj = ccrs.LambertConformal(central_longitude=-95,central_latitude=35)

#u and v winds
u = rethinnedData.eastward_wind.values * units("knots")
v = rethinnedData.northward_wind.values * units("knots")
gust = rethinnedData.wind_gust.values * units("knots")
mpcalc.wind_direction(u,v)
wdir = mpcalc.wind_direction (u,v)
ug, vg = mpcalc.wind_components(gust,wdir)



#create figure
fig = plt.figure(figsize=(10,10),dpi=100)
ax = fig.add_subplot(1,1,1,projection=proj)

#add elements to make map more useable
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.STATES)
ax.add_feature(cfeature.BORDERS)

#zoom on the Northern Plains
ax.set_extent((-105, -90, 40, 50))


#create stationplot (will actually need to create 2 to handle wind gusts)
stationPlots = StationPlot (ax, rethinnedData["longitude"], rethinnedData["latitude"], transform = ccrs.PlateCarree())
stationPlots_Gust = StationPlot (ax, rethinnedData["longitude"], rethinnedData["latitude"], transform = ccrs.PlateCarree())

#temperature in red
stationPlots.plot_parameter ((-1,1), rethinnedData["air_temperature"].values, color = 'red')
#dewpoint in dark green
stationPlots.plot_parameter ((-1,-1), rethinnedData["dew_point_temperature"].values, color = 'darkgreen')
#pressure (need to format to be the final 3 digits)
stationPlots.plot_parameter ((1,1),rethinnedData["air_pressure_at_sea_level"].values, color = "black",
                             formatter = lambda p: format (10*p, ".0f")[-3:])
#station ID in dark blue
stationPlots.plot_text ((1.5,-1), rethinnedData["station_id"], color = 'blue')
#visibility in black (need to convert from meters to miles)
vis = rethinnedData["visibility"].values * units("m")
vis = vis.to("miles")
stationPlots.plot_parameter ((-2,0), rethinnedData["visibility"].values, color = 'black')
#wind gust in red
stationPlots_Gust.plot_barb (ug,vg, color = "red")
#wind in black
stationPlots.plot_barb (rethinnedData["eastward_wind"].values, rethinnedData["northward_wind"].values)
#cloud cover
stationPlots.plot_symbol ((0,0), rethinnedData["cloud_coverage"].values, sky_cover)
#current weather
stationPlots.plot_symbol ((-1,0), rethinnedData["current_wx1_symbol"].values, current_weather)
#save the map as surface_obs.png
plt.savefig ("surface_obs.png")

AttributeError: 'DataFrame' object has no attribute 'eastward_wind'